<a href="https://colab.research.google.com/github/bijayabc/mBERT-finetuning/blob/main/lg_BERT_last_two_layers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install watermark transformers datasets lightning

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 824.2/824.2 kB 50.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 132.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 98.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 58.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 42.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 107.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
%load_ext watermark
%watermark --conda -p torch,transformers,datasets,lightning

torch       : 2.6.0+cu124
transformers: 4.55.1
datasets    : 4.0.0
lightning   : 2.5.3

conda environment: n/a



In [ ]:
from datasets import load_dataset
import lightning as L
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification

import numpy as np
import pandas as pd
import torch

# Load and clean the dataset

In [ ]:
dataset = load_dataset('IRIIS-RESEARCH/sentiment-Analysis-Nepali')
print(dataset)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/419 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/5.88M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/1.47M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/65106 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/16279 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['sentences', 'sentiment'],
        num_rows: 65106
    })
    test: Dataset({
        features: ['sentences', 'sentiment'],
        num_rows: 16279
    })
})


In [ ]:
dataset_raw = dataset['train']
split = dataset_raw.train_test_split(test_size=0.2, seed=42)
train_dataset_raw = split['train']
val_dataset_raw = split['test']
test_dataset_raw = dataset['test']

Tokenize the dataset

In [ ]:
# initialize the bert tokenizer
tokenizer = AutoTokenizer.from_pretrained('bert-base-multilingual-cased')

def tokenize_function(examples):
    texts = [str(t) if t is not None else "" for t in examples['sentences']]
    return tokenizer(texts, padding='max_length', truncation=True, max_length=128)

# Tokenize each split separately
train_dataset = train_dataset_raw.map(tokenize_function, batched=True)
val_dataset = val_dataset_raw.map(tokenize_function, batched=True)
test_dataset = test_dataset_raw.map(tokenize_function, batched=True)

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

Map:   0%|          | 0/52084 [00:00<?, ? examples/s]

Map:   0%|          | 0/13022 [00:00<?, ? examples/s]

Map:   0%|          | 0/16279 [00:00<?, ? examples/s]

In [ ]:
for dataset in [train_dataset, val_dataset, test_dataset]:
  dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'sentiment'])

Set up Dataloaders

In [ ]:
from torch.utils.data import DataLoader, Dataset

# try using num_workers=2 and see if it is faster
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=12, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=64, num_workers=12)
test_loader = DataLoader(test_dataset, batch_size=64, num_workers=12)

for batch in train_loader:
    input_ids = batch['input_ids']
    attention_mask = batch['attention_mask']
    labels = batch['sentiment']
    # feed these into your model

In [ ]:
for dataset in [train_dataset, val_dataset, test_dataset]:
  print(len(dataset))

52084
13022
16279


# Initialize the Model

In [ ]:
# Initialize a BERT model for text classification
model_name = 'bert-base-multilingual-cased'
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Freeze all layers

In [ ]:
for param in model.parameters():
    param.requires_grad = False

In [ ]:
model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1

Unfreeze some layers

In [ ]:
# Unfreeze the last two layers
for layer in model.bert.encoder.layer[-2:]:
  for param in layer.parameters():
    param.requires_grad = True

# Unfreeze classifier
for param in model.classifier.parameters():
    param.requires_grad = True

In [ ]:
def count_parameters(model):
  return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"Total trainable parameters: {count_parameters(model):,}")

Total trainable parameters: 14,177,282


# Finetuning

Wrap in LightningModule for training

In [ ]:
import lightning as L
import torch
import torchmetrics

class CustomLightningModule(L.LightningModule):
  def __init__(self, model, learning_rate=4e-5, weight_decay=0.01):
    super().__init__()
    self.model = model
    self.learning_rate = learning_rate
    self.weight_decay = weight_decay

    self.val_acc = torchmetrics.Accuracy(task="multiclass", num_classes=2)
    self.test_acc = torchmetrics.Accuracy(task='multiclass', num_classes=2)

  def forward(self, input_ids, attention_mask, labels):
    return self.model(input_ids, attention_mask=attention_mask, labels=labels)

  def training_step(self, batch, batch_idx):
    outputs = self(
        input_ids=batch["input_ids"],
        attention_mask=batch["attention_mask"],
        labels=batch["sentiment"]
    )
    self.log("train_loss", outputs.loss, on_step=False, on_epoch=True, prog_bar=True)
    return outputs.loss # This is passed to the optimizer for training

  def validation_step(self, batch, batch_idx):
    outputs = self(
        input_ids=batch["input_ids"],
        attention_mask=batch["attention_mask"],
        labels=batch["sentiment"]
    )
    self.log("val_loss", outputs["loss"], on_step=False, on_epoch=True, prog_bar=True)

    logits = outputs['logits']
    predicted_labels = torch.argmax(logits, dim=1)
    self.val_acc(predicted_labels, batch['sentiment'])
    self.log('val_acc', self.val_acc, on_step=False, on_epoch=True, prog_bar=True)

  def test_step(self, batch, batch_idx):
    outputs = self(
        input_ids=batch["input_ids"],
        attention_mask=batch["attention_mask"],
        labels=batch["sentiment"]
    )
    logits = outputs['logits']
    predicted_labels = torch.argmax(logits, dim=1)
    self.test_acc(predicted_labels, batch['sentiment'])
    self.log('accuracy', self.test_acc, on_step=False, on_epoch=True, prog_bar=True)

  def configure_optimizers(self):
    # optimizer = torch.optim.Adam(self.parameters(), lr=self.learning_rate)
    optimizer = torch.optim.AdamW(self.parameters(), lr=self.learning_rate, weight_decay=self.weight_decay)
    return optimizer

lightning_model = CustomLightningModule(model)

In [ ]:
from lightning.pytorch.callbacks import ModelCheckpoint, EarlyStopping
from lightning.pytorch.loggers import CSVLogger

callbacks = [
    ModelCheckpoint(
        save_top_k=1, mode="max", monitor="val_acc"
    ),  # save top 1 model
    EarlyStopping(
        monitor="val_acc",
        mode="max",
        patience=1         # stop if val_acc doesn’t improve after 1 epoch
    )
]
logger = CSVLogger(save_dir='logs/', name='my-model')

In [ ]:
trainer = L.Trainer(
    max_epochs=5,
    callbacks=callbacks,
    accelerator="gpu",
    precision="16-mixed",
    devices=[0],
    logger=logger,
    log_every_n_steps=10,
    gradient_clip_val=1.0,
)

INFO: Using 16bit Automatic Mixed Precision (AMP)
INFO:lightning.pytorch.utilities.rank_zero:Using 16bit Automatic Mixed Precision (AMP)
INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [ ]:
import time
start = time.time()

trainer.fit(model=lightning_model,
            train_dataloaders=train_loader,
            val_dataloaders=val_loader)

end = time.time()
elapsed = end - start
print(f'time elapsed: {elapsed/60:.2f} min')

INFO: You are using a CUDA device ('NVIDIA A100-SXM4-40GB') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
INFO:lightning.pytorch.utilities.rank_zero:You are using a CUDA device ('NVIDIA A100-SXM4-40GB') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.11/dist-packages/lightning/pytorch/utilities/model_summary/model_summary.

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

time elapsed: 1.66 min


In [ ]:
trainer.test(lightning_model, dataloaders=train_loader, ckpt_path="best")

INFO: Restoring states from the checkpoint path at logs/my-model/version_0/checkpoints/epoch=2-step=2442.ckpt
INFO:lightning.pytorch.utilities.rank_zero:Restoring states from the checkpoint path at logs/my-model/version_0/checkpoints/epoch=2-step=2442.ckpt
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO: Loaded model weights from the checkpoint at logs/my-model/version_0/checkpoints/epoch=2-step=2442.ckpt
INFO:lightning.pytorch.utilities.rank_zero:Loaded model weights from the checkpoint at logs/my-model/version_0/checkpoints/epoch=2-step=2442.ckpt
/usr/local/lib/python3.11/dist-packages/lightning/pytorch/trainer/connectors/data_connector.py:484: Your `test_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for val/test dataloaders.


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         accuracy          │    0.9343752264976501     │
└───────────────────────────┴───────────────────────────┘

[{'accuracy': 0.9343752264976501}]

In [ ]:

trainer.test(lightning_model, dataloaders=val_loader, ckpt_path="best")

INFO: Restoring states from the checkpoint path at logs/my-model/version_0/checkpoints/epoch=2-step=2442.ckpt
INFO:lightning.pytorch.utilities.rank_zero:Restoring states from the checkpoint path at logs/my-model/version_0/checkpoints/epoch=2-step=2442.ckpt
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO: Loaded model weights from the checkpoint at logs/my-model/version_0/checkpoints/epoch=2-step=2442.ckpt
INFO:lightning.pytorch.utilities.rank_zero:Loaded model weights from the checkpoint at logs/my-model/version_0/checkpoints/epoch=2-step=2442.ckpt


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         accuracy          │    0.8134695291519165     │
└───────────────────────────┴───────────────────────────┘

[{'accuracy': 0.8134695291519165}]

In [ ]:
trainer.test(model=lightning_model, dataloaders=test_loader, ckpt_path=None)

INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         accuracy          │    0.8186006546020508     │
└───────────────────────────┴───────────────────────────┘

[{'accuracy': 0.8186006546020508}]